In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import curdoc
from bokeh.models import Slider, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.layouts import column, gridplot, layout
from scipy.integrate import odeint


sp.init_printing()

poetry run bokeh serve --show .\Lab9.ipynb 

N - całkowita populacja;
I0 - początkowa liczba zarażonych;
R0 - liczba ozdrowiałych;
S0 - potencjalni zarażeni;
beta - Wzkaźnik kontaktu;
gamma - współczynnik ozdrowień
t - zakres czasu w dniach

In [2]:
N = 1000
I0, R0 = 1, 0
S0 = N - I0 - R0
beta, gamma = 0.2, 1./10 
t = np.linspace(0, 260, 260)

Równanie

In [3]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

Vektor kondycji

In [4]:
y0 = S0, I0, R0
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

Tworzenie wykresu

In [5]:
source = ColumnDataSource(data=dict(x=t, y1=S,y2=I,y3=R))
fig = figure(height=800, width=800, title="SIR model", tools="crosshair,pan,reset,save,wheel_zoom", x_range=[0,200], y_range=[0, N])
fig.line('x', 'y1', source=source, line_width=2, line_alpha=0.6, color ='blue', legend_label = 'S - Podejrzany')
fig.line('x', 'y2', source=source, line_width=2, line_alpha=0.6, color ='red', legend_label = 'I - Zainfekowany')
fig.line('x', 'y3', source=source, line_width=2, line_alpha=0.6, color ='green', legend_label = 'R - Odporny ozdrowieniec')

GlyphRenderer(id='p1101', ...)

Slidery i ich ustawienia

In [6]:
beta_slide = Slider(title="Wskaźnik kontaktu", value=0.0, start=0.0, end=1.0, step=0.01)
gamma_slide = Slider(title="Współczynnik ozdrowień", value=0.0, start=0.0, end=.5, step=0.005)

In [7]:
def update_data(attrname, old, new):

    beta = beta_slide.value
    gamma = gamma_slide.value
    
    y0 = S0, I0, R0

    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    source.data = dict(x = t, y1 = S, y2 = I, y3 = R)

Opisanie wykresu i sliderów

In [8]:
for w in [gamma_slide, beta_slide]:
    w.on_change('value', update_data)

curdoc().add_root(column(beta_slide,gamma_slide,fig))
curdoc().title = "Model SIR"